# Structuring programs using types and dispatch

Previously we specified an algorithm directly using a function. In many Julia packages it is common to use *dispatch* to do this. 

In Julia, **dispatch** refers to choosing which **method** (version) of a function to use, according to the type of the arguments. (**Multiple dispatch** is when the types of several different arguments are involved.)

Let's define some types to represent different differentiation methods.

In [5]:
struct Dual 
    v::Float64
    d::Float64
end

In [6]:
methods(Dual)

# 2 methods for generic function "(::Type)":
[1] Dual(v::Float64, d::Float64) in Main at In[5]:2
[2] Dual(v, d) in Main at In[5]:2

#### Exercise 1

1. Define an abstract type `DifferentiationAlgorithm`.


2. Define subtypes `FiniteDifference`, `MyAutoDiff` (for our implementation) and `AutoDiff` (for the `ForwardDiff` implementation).


3. Implement the function `derivative(f, x, algorithm)` using **dispatch**: for each of the three types, define a version of this function in which `algorithm` is specified to be of that type by using the type annotation operator `::`.


4. Verify that these work by writing tests for them.

You could do:

In [4]:
function derivative(f, x, algorithm)
    if algorithm == :newton
        ...
    elseif algorithm == :bisection
        ...
    elseif
        ...
    end
end

LoadError: syntax: invalid identifier name "..."

In [1]:
abstract type DifferentiationAlgorithm end

In [2]:
struct FiniteDifference <: DifferentiationAlgorithm 
    h::Float64
end

In [9]:
struct AutoDiff <: DifferentiationAlgorithm end

In [7]:
finite_difference(f, a, h) = (f(a + h) - f(a - h)) / (2h)

finite_difference (generic function with 1 method)

In [8]:
using ForwardDiff
forwarddiff(f, x) = ForwardDiff.derivative(f, x)

forwarddiff (generic function with 1 method)

In [10]:
function derivative(f, x, algorithm::AutoDiff)
    return forwarddiff(f, x)
end

derivative (generic function with 1 method)

In [12]:
g(x) = x^2 - 2
a = 3.0

3.0

In [13]:
derivative(g, a, AutoDiff)

MethodError: MethodError: no method matching derivative(::typeof(g), ::Float64, ::Type{AutoDiff})
Closest candidates are:
  derivative(::Any, ::Any, !Matched::AutoDiff) at In[10]:2

In [14]:
typeof(AutoDiff)

DataType

In [15]:
autodiff = AutoDiff()

AutoDiff()

In [16]:
typeof(autodiff)

AutoDiff

In [17]:
derivative(g, a, autodiff)

6.0

In [18]:
derivative(g, a, AutoDiff())

6.0

In [19]:
function derivative(f, x, algorithm::FiniteDifference)
    h = algorithm.h
    return finite_difference(f, x, h)
end

derivative (generic function with 2 methods)

In [22]:
derivative(g, a, FiniteDifference())

MethodError: MethodError: no method matching FiniteDifference()
Closest candidates are:
  FiniteDifference(!Matched::Float64) at In[2]:2
  FiniteDifference(!Matched::Any) at In[2]:2

In [23]:
derivative(g, a, FiniteDifference(0.01))

5.999999999999872

In [25]:
algorithm = FiniteDifference(0.01)

FiniteDifference(0.01)

In [26]:
typeof(algorithm)

FiniteDifference

In [27]:
algorithm.h

0.01

In [30]:
fieldnames(typeof(algorithm))

(:h,)

In [31]:
FiniteDifference()

MethodError: MethodError: no method matching FiniteDifference()
Closest candidates are:
  FiniteDifference(!Matched::Float64) at In[2]:2
  FiniteDifference(!Matched::Any) at In[2]:2

In [32]:
methods(FiniteDifference)

# 2 methods for generic function "(::Type)":
[1] FiniteDifference(h::Float64) in Main at In[2]:2
[2] FiniteDifference(h) in Main at In[2]:2

In [33]:
FiniteDifference() = FiniteDifference(0.001)

FiniteDifference

In [34]:
methods(FiniteDifference)

# 3 methods for generic function "(::Type)":
[1] FiniteDifference() in Main at In[33]:1
[2] FiniteDifference(h::Float64) in Main at In[2]:2
[3] FiniteDifference(h) in Main at In[2]:2

In [35]:
FiniteDifference()

FiniteDifference(0.001)

In [36]:
derivative(g, a, FiniteDifference() )

5.999999999999339

In [37]:
methods(derivative)

# 2 methods for generic function "derivative":
[1] derivative(f, x, algorithm::AutoDiff) in Main at In[10]:2
[2] derivative(f, x, algorithm::FiniteDifference) in Main at In[19]:2

#### Exercise 2

1. Write a version of the Newton algorithm that takes an optional keyword argument `algorithm` specifying the differentiation algorithm to use, and which has a default value of `AutoDiff`.

In [42]:
function newton(f, x0, n=10; 
                    algorithm::DifferentiationAlgorithm=AutoDiff())
    
    df = x -> derivative(f, x, algorithm)
    
#     newton(f, df, x0)
    
end

newton (generic function with 2 methods)

In [43]:
newton(g, a)

6.0

## Functions as types

How could we differentiate the `sin` function? Of course, we can do it using e.g. finite differences, but in cases like this we actually know the exact, analytical derivative, in this case `cos`, i.e $\sin'(x) = \cos(x)$.
Is there are a way to tell Julia to use this directly? I.e. if `f` is equal to the `sin` function, then we should make a special version of our `derivative` function.

It turns out that there is, using dispatch, by checking what *type* the `sin` function has:

#### Exercise 3

1. Use `typeof` to find the type of the `sin` function.


2. Use this to make a special dispatch for `derivative(sin, x)`.

In [44]:
typeof(sin)

typeof(sin)

In [45]:
derivative(::typeof(sin), x) = cos(x)

derivative (generic function with 3 methods)

The package [`ChainRules.jl`](https://github.com/JuliaDiff/ChainRules.jl) contains definitions like this and is used inside `ForwardDiff.jl` and other packages that need to know the derivatives of functions.

## Representing a problem using a type

A root-finding problem requires several pieces of information: a function, a starting point, a root-finding algorithm to use, possibly a derivative, etc. We could just pass all of these as arguments to a function.
An alternative is to wrap up the various pieces of information into a new composite type.

#### Exercise 4

1. Make a `RootAlgorithm` abstract type.


2. Make a `Newton` subtype.


3. Make `Newton` a *callable* type using the syntax

    ```
    function (algorithm::Newton)(x)
        ...
     end
     
     ```
    
    This means that you will be able to call an object of type `Newton` as if it were a standard function, by passing in an argument. (You can add further arguments as necessary.)


4. Make a `RootProblem` type that contains the necessary information for a root problem. Do not specify types for the fields in this type yet. One of the fields should be called `algorithm`. 


5. Make a function `solve` that calls the field `algorithm` as a function.

In [46]:
struct MyNewton
end

In [48]:
mynewton = MyNewton()

MyNewton()

In [52]:
function (f::MyNewton)(x)
    return 3x
end

In [53]:
mynewton(10)

30

Make FiniteDifference into a callable thing:

In [54]:
fd = FiniteDifference()

FiniteDifference(0.001)

If I try to treat `fd` as a function, what happens?

In [55]:
fd(10)

MethodError: MethodError: objects of type FiniteDifference are not callable

Tell Julia how to treat it as a function:

In [56]:
function (fd::FiniteDifference)(f, x)
    h = fd.h
    finite_difference(f, x, h)
end

In [57]:
fd(g, a)

5.999999999999339

In [58]:
abstract type RootAlgorithm end

In [59]:
struct Newton <: RootAlgorithm end

In [71]:
(n::Newton)(args...) = ( @show args; newton(args...) )

In [67]:
n = Newton()
methods(Newton)

# 1 method for generic function "(::Type)":
[1] Newton() in Main at In[59]:1

In [72]:
newt = Newton()

newt(g, a)

args = (g, 3.0)


6.0

In [73]:
struct RootProblem
    f
    x0
    algorithm::RootAlgorithm
end

In [81]:
function solve(prob::RootProblem)
    prob.algorithm(prob.f, prob.x0)
end

solve (generic function with 1 method)

In [82]:
prob = RootProblem(g, a, Newton())

RootProblem(g, 3.0, Newton())

In [83]:
solve(prob)

args = (g, 3.0)


6.0

### Type-based dispatch

So far we are not using the types to their full advantage: we wish to *dispatch* on the *type* of `algorithm`. We can do so by **parametrising the type**:

In [1]:
struct RootProblem2{T<:RootAlgorithm}
    ...
    algorithm::T
end

When we create an object of type `RootProblem2`, we will get a specialised version with the correct type. We can now use that in dispatch:

In [ ]:
solve(prob::RootProblem2{Newton}) = ...

In [ ]:
solve(prob::RootProblem2{Bisection}) = ...

In [ ]:
struct RootProblem2{F, X<:Real, T<:RootAlgorithm}
    f::F
    x0::X
    algorithm::T
end

#### Exercise 5

1. Implement this.


2. Put everything together to be able to solve a root problem using a particular derivative algorithm.

#### Exercise 6

1. Implement a `MultipleRootProblem` type that specifies an interval over which we would like to find all roots.


2. Write a simple implementation of the algorithm using multiple starting points in the interval and making a list of unique roots found by that procedure.


3. Load the `Polynomials.jl` package and write a dispatch that specialises on polynomials and calls the root finder in that package.

In [85]:
using Polynomials

In [86]:
p = Poly([1, 2, 3])

Poly(1 + 2*x + 3*x^2)

In [87]:
p(10)

321

In [89]:
p isa Function

false

## Other uses of types

Other examples of different usages of types include:

- [`ModelingToolkit.jl`](https://github.com/JuliaDiffEq/ModelingToolkit.jl)

    Types are introduced to represent variables and operations. In this way it is relatively simple to build up a way to output symbolic expressions from standard Julia functions.
    
    
- https://github.com/MikeInnes/diff-zoo defines types to represent "tapes" recording sequences of operations. This is a precursor to tools such as [`Zygote.jl`](https://github.com/FluxML/Zygote.jl), which performs advanced automatic differentiation on code at a lower level.

### Traits

An important use of types that we have not addressed here is to define **traits**. These are labels that can be assigned to different types that may then be dispatched on, even if those types are not in a Julia type hierarchy.

See e.g. the implementation in [`SimpleTraits.jl`](https://github.com/mauro3/SimpleTraits.jl).